# Human Activity Recognition - GNP Featurized

Last Update : 21 July 2019

In [1]:
N_THREADS = 8
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = False # can also be set in the trainer

## Environment

In [2]:
cd ..

/master


In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""

import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import pandas as pd
import h5py


import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


# Dataset

In [4]:
from skssl.transformers.neuralproc.datasplit import CntxtTrgtGetter, GetRandomIndcs, get_all_indcs
from utils.data.tsdata import get_timeseries_dataset, SparseMultiTimeSeriesDataset


get_cntxt_trgt_test = CntxtTrgtGetter(contexts_getter=GetRandomIndcs(min_n_indcs=0.1, max_n_indcs=0.5),
                                     targets_getter=get_all_indcs,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt_feat = CntxtTrgtGetter(contexts_getter=get_all_indcs,
                                     targets_getter=get_all_indcs,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes


get_cntxt_trgt = CntxtTrgtGetter(contexts_getter=GetRandomIndcs(min_n_indcs=0.01, max_n_indcs=0.5),
                                 targets_getter=GetRandomIndcs(min_n_indcs=0.5, max_n_indcs=0.99),
                                 is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

In [6]:
import random

data_train = get_timeseries_dataset("har")(split="train")
data_test = get_timeseries_dataset("har")(split="test")
data_both = get_timeseries_dataset("har")(split="both")

def cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=False):
    def mycollate(batch):
        min_length = min([v.size(0) for b in batch for k,v in b[0].items() if "X" in k])
        # chose first min_legth of each (assumes that randomized)
        
        batch = [({k:v[:min_length, ...] for k,v in b[0].items()}, b[1]) for b in batch]        
        collated = torch.utils.data.dataloader.default_collate(batch)
        
        X = collated[0]["X"]
        y = collated[0]["y"]
        
        if is_repeat_batch:
            
            X = torch.cat([X,X], dim=0)
            y = torch.cat([y,y], dim=0)
        
        collated[0]["X"], collated[0]["y"], collated[0]["X_trgt"], collated[0]["y_trgt"] = get_cntxt_trgt(X, y)
        
        return collated
    return mycollate

In [7]:
X_DIM = 1  # 1D spatial input (although actually 2 but the first is for sparse channels)
Y_DIM = data_train.data.shape[-1] # multiple channels
N_TARGETS = len(np.unique(data_train.targets))

## Model

In [8]:
import torch.nn as nn
from skssl.transformers import GlobalNeuralProcess, NeuralProcessLoss, AttentiveNeuralProcess
from skssl.utils.helpers import rescale_range
from skssl.predefined import UnetCNN, CNN, MLP, SparseSetConv, SetConv, MlpRBF, GaussianRBF, BatchSparseSetConv
from skssl.transformers.neuralproc.datasplit import precomputed_cntxt_trgt_split
from utils.helpers import count_parameters

In [9]:
from copy import deepcopy

models = {}

unet = partial(UnetCNN,
               Conv=torch.nn.Conv1d,
               Pool=torch.nn.MaxPool1d,
               upsample_mode="linear",
               n_layers=18,
               is_double_conv=True,
               is_depth_separable=True,
               Normalization=torch.nn.BatchNorm1d,
               is_chan_last=True,
               bottleneck=None,
               kernel_size=7,
               max_nchannels=256)

kwargs = dict(x_dim=X_DIM, 
              y_dim=Y_DIM,
              min_std=5e-3,
                n_tmp_queries=128,
                r_dim=64,
              keys_to_tmp_attn=partial(SetConv, RadialBasisFunc=GaussianRBF),
              TmpSelfAttn=unet,
              tmp_to_queries_attn=partial(SetConv, RadialBasisFunc=GaussianRBF),
              is_skip_tmp=False,
              is_use_x=False,
              get_cntxt_trgt=precomputed_cntxt_trgt_split,
              is_encode_xy=False)

models["transformer_gnp_large"] = partial(GlobalNeuralProcess, **kwargs)

kwargs_bis = deepcopy(kwargs)
kwargs_bis["TmpSelfAttn"] = partial(unet, n_layers=14, bottleneck=True)

models["transformer_gnp_bottleneck"] = partial(GlobalNeuralProcess, **kwargs_bis)

In [10]:
from utils.helpers import count_parameters
for k,v in models.items():
    print(k, "- N Param:", count_parameters(v()))

transformer_gnp_large - N Param: 1006936
transformer_gnp_bottleneck - N Param: 665944


# Training

In [11]:
from ntbks_helpers import train_models_
from skorch.dataset import CVSplit

In [12]:
N_EPOCHS = 100 
BATCH_SIZE = 64
IS_RETRAIN = False # if false load precomputed
chckpnt_dirname="results/challenge/har/"

In [13]:
data_trainers = {}
data_percentages =  [0.05, 0.1, 0.3, 0.5, 0.7, 1]
datas = {p:get_timeseries_dataset("har")(split="both", data_perc=p, is_fill_mean=False)
         for p in data_percentages}

In [12]:
for p,data in datas.items():
    data_trainers.update(train_models_({"{}%har".format(int(p*100)): data}, 
                              models, 
                              NeuralProcessLoss,
                              chckpnt_dirname=chckpnt_dirname,
                              max_epochs=N_EPOCHS,
                              batch_size=BATCH_SIZE,
                              is_retrain=IS_RETRAIN,
                              train_split=CVSplit(0.05),
                              iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt),  
                              iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_test),
                              mode="transformer"))


--- Loading 5%har/transformer_gnp_large ---

5%har/transformer_gnp_large best epoch: 63 val_loss: -1.2570536011631048

--- Loading 5%har/transformer_gnp_bottleneck ---

5%har/transformer_gnp_bottleneck best epoch: 64 val_loss: -1.3726283962286792

--- Loading 10%har/transformer_gnp_large ---

10%har/transformer_gnp_large best epoch: 48 val_loss: -1.1501760788334225

--- Loading 10%har/transformer_gnp_bottleneck ---

10%har/transformer_gnp_bottleneck best epoch: 39 val_loss: -0.18719741117607042

--- Loading 30%har/transformer_gnp_large ---

30%har/transformer_gnp_large best epoch: 66 val_loss: -3.663724695536697

--- Loading 30%har/transformer_gnp_bottleneck ---

30%har/transformer_gnp_bottleneck best epoch: 54 val_loss: -3.0089645237598606

--- Loading 50%har/transformer_gnp_large ---

50%har/transformer_gnp_large best epoch: 65 val_loss: -4.738986269247184

--- Loading 50%har/transformer_gnp_bottleneck ---

50%har/transformer_gnp_bottleneck best epoch: 40 val_loss: -2.27140642184655

# Featurizing

In [13]:
def save_transformed_data(chckpnt_dirname, data_trainers, datas):
    for k, trainer in data_trainers.items():
        perc = float(k.split("%")[0]) / 100
        model_name = k.split("/")[1]
        data = datas[perc]
        trainer.set_params(iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt_feat),
                           iterator_valid__shuffle=False) # make sure not shuffling because only transforming X
        transformed_data = trainer.transform(data)
        np.save(chckpnt_dirname+k+"/transformed_data.npy", transformed_data, allow_pickle=False)

In [14]:
save_transformed_data(chckpnt_dirname, data_trainers, datas)
# saving all transformed data

# Evaluation

In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [16]:
import torch.nn as nn
from skssl.predefined import RNN, MLP


In [17]:
class FeatureMLP(nn.Module):

    def __init__(self, feat_size, y_dim, output_size, **kwargs):
        super().__init__()
        self.out = MLP(feat_size+y_dim*4, output_size, **kwargs)

    def forward(self, X, X_feat, y=None):

        if y is not None:
            # is there's y then that's the actual input and X is time
            X = y

        outputs = self.out(torch.cat([X_feat, X.mean(-2), X.max(-2)[0], X.min(-2)[0], X.var(-2)], dim=-1))
        return outputs

In [18]:
models = dict(mlp_hand = partial(FeatureMLP, feat_size=256, y_dim=Y_DIM, output_size=N_TARGETS, hidden_size=128, 
                dropout=0.5, n_hidden_layers=3, is_res=True),
             )

In [19]:
from utils.helpers import count_parameters
for k,v in models.items():
    print(k, "- N Param:", count_parameters(v()))

mlp_hand - N Param: 71302


In [20]:
from skorch.dataset import Dataset

In [21]:
def train_eval_on_featurized(X_train, X_test, Y_train, Y_test):
    mlp = MLPClassifier(solver="lbfgs", hidden_layer_sizes=(128,))
    mlp = SVC()
    mlp.fit(X_train, Y_train)
    print(mlp.score(X_test, Y_test))

In [22]:
data_trainers_feat = {}

for k, trainer in data_trainers.items():
    perc = float(k.split("%")[0]) / 100
    n_train = datas[perc].n_train
    model_name = k.split("/")[1]
    y = datas[perc].targets
    transformed_data = np.load(chckpnt_dirname+k+"/transformed_data.npy", allow_pickle=False)
    X_train, X_test, Y_train, Y_test = transformed_data[:n_train], transformed_data[n_train:], y[:n_train].squeeze(), y[n_train:].squeeze()
    #train_eval_on_featurized(X_train, X_test, Y_train, Y_test)
    
                                                
    data_trainers_feat.update(train_models_({"{}%har".format(int(perc*100)): (Dataset(X={"X_feat":X_train.astype(np.float32), 
                                                                   "X":datas[perc].data[:n_train].astype(np.float32)}, 
                                                                y=torch.from_numpy(Y_train).long()), 
                                                        Dataset(X={"X_feat":X_test.astype(np.float32), 
                                                                   "X":datas[perc].data[n_train:].astype(np.float32)}, 
                                                                y=torch.from_numpy(Y_test).long()))}, 
                                               {model_name +"/feat_"+ name: partial(m, feat_size=X_train.shape[-1]) for name,m in models.items()}, 
                                                 chckpnt_dirname=chckpnt_dirname,
                                                  max_epochs=N_EPOCHS,
                                                  batch_size=BATCH_SIZE,
                                                 is_retrain=False))
    
    
# 100%: MLP 0.8235 (no feat)
# 
    


--- Loading 5%har/transformer_gnp_large/feat_mlp_hand ---

5%har/transformer_gnp_large/feat_mlp_hand best epoch: 25 val_loss: 0.3994055680594365

--- Loading 5%har/transformer_gnp_bottleneck/feat_mlp_hand ---

5%har/transformer_gnp_bottleneck/feat_mlp_hand best epoch: 27 val_loss: 0.40084968634205104

--- Loading 10%har/transformer_gnp_large/feat_mlp_hand ---

10%har/transformer_gnp_large/feat_mlp_hand best epoch: 17 val_loss: 0.40527173914179626

--- Loading 10%har/transformer_gnp_bottleneck/feat_mlp_hand ---

10%har/transformer_gnp_bottleneck/feat_mlp_hand best epoch: 12 val_loss: 0.40299248517388847

--- Loading 30%har/transformer_gnp_large/feat_mlp_hand ---

30%har/transformer_gnp_large/feat_mlp_hand best epoch: 24 val_loss: 0.2857550858721595

--- Loading 30%har/transformer_gnp_bottleneck/feat_mlp_hand ---

30%har/transformer_gnp_bottleneck/feat_mlp_hand best epoch: 25 val_loss: 0.26946120105195864

--- Loading 50%har/transformer_gnp_large/feat_mlp_hand ---

50%har/transformer_gn

In [23]:
for k,t in data_trainers_feat
.items():
    for e, h in enumerate(t.history[::-1]):
        if h["valid_loss_best"]:
            print(k, "epoch:", len(t.history)-e, 
                  "val_loss:", h["valid_loss"], 
                  "val_acc:", h["valid_acc"])
            break

5%har/transformer_gnp_large/feat_mlp_hand epoch: 25 val_loss: 0.3994055680594365 val_acc: 0.845945028842891
5%har/transformer_gnp_bottleneck/feat_mlp_hand epoch: 27 val_loss: 0.40084968634205104 val_acc: 0.8500169664065151
10%har/transformer_gnp_large/feat_mlp_hand epoch: 17 val_loss: 0.40527173914179626 val_acc: 0.833050559891415
10%har/transformer_gnp_bottleneck/feat_mlp_hand epoch: 12 val_loss: 0.40299248517388847 val_acc: 0.842212419409569
30%har/transformer_gnp_large/feat_mlp_hand epoch: 24 val_loss: 0.2857550858721595 val_acc: 0.9131319986426875
30%har/transformer_gnp_bottleneck/feat_mlp_hand epoch: 25 val_loss: 0.26946120105195864 val_acc: 0.9117746861214795
50%har/transformer_gnp_large/feat_mlp_hand epoch: 3 val_loss: 0.20970985878885934 val_acc: 0.9097387173396675
50%har/transformer_gnp_bottleneck/feat_mlp_hand epoch: 23 val_loss: 0.2706818469007783 val_acc: 0.9161859518154055
70%har/transformer_gnp_large/feat_mlp_hand epoch: 14 val_loss: 0.2614200467937147 val_acc: 0.90702409

# Evaluation SSL

In [ ]:
data_trainers_feat = {}


perc = 1#float(k.split("%")[0]) / 100
n_train = datas[perc].n_train
model_name = k.split("/")[1]
y = datas[perc].targets
transformed_data = np.load(chckpnt_dirname+k+"/transformed_data.npy", allow_pickle=False)
X_train, X_test, Y_train, Y_test = transformed_data[:n_train], transformed_data[n_train:], y[:n_train].squeeze(), y[n_train:].squeeze()
#train_eval_on_featurized(X_train, X_test, Y_train, Y_test)


data_trainers_feat.update(train_models_({"{}%har".format(int(perc*100)): (Dataset(X={"X_feat":X_train.astype(np.float32), 
                                                               "X":datas[perc].data[:n_train].astype(np.float32)}, 
                                                            y=torch.from_numpy(Y_train).long()), 
                                                    Dataset(X={"X_feat":X_test.astype(np.float32), 
                                                               "X":datas[perc].data[n_train:].astype(np.float32)}, 
                                                            y=torch.from_numpy(Y_test).long()))}, 
                                           {model_name +"/feat_"+ name: partial(m, feat_size=X_train.shape[-1]) for name,m in models.items()}, 
                                             chckpnt_dirname=chckpnt_dirname,
                                              max_epochs=N_EPOCHS,
                                              batch_size=BATCH_SIZE,
                                             is_retrain=False))
    
    

    

In [15]:
models.keys()

dict_keys(['transformer_gnp_large', 'transformer_gnp_bottleneck'])

In [14]:
from utils.data.helpers import make_ssl_dataset_

In [17]:
perc = 1#float(k.split("%")[0]) / 100
transformed_data = np.load(chckpnt_dirname+"{}%har/".format(int(perc*100))+'transformer_gnp_large'+"/transformed_data.npy", allow_pickle=False)
data = deepcopy(datas[1])
data.train = transformed_data
data_train = make_ssl_dataset_()

NameError: name 'perc' is not defined

In [ ]:
data_trainers = {}
data_perc = 1

for label_perc in label_percentages:
    data_train, _, data_test = get_train_dev_test_ssl("har", n_labels=label_perc, data_perc=data_perc, dev_size=0)
    
    data_trainers.update(train_models_({"{}%har_{}%lab".format(int(data_perc*100), int(label_perc*100)): (data_train, data_test)}, 
                         models, 
                         chckpnt_dirname=chckpnt_dirname,
                          max_epochs=N_EPOCHS,
                          batch_size=64,
                         is_retrain=True, 
                         iterator_train=get_supervised_iterator))

In [ ]:
eg = 3000
data = data_train
n_targets = 60

idcs = torch.randperm(128)[:n_targets]
X_cntxt = data[eg][0]["X"][idcs].unsqueeze(0)
Y_cntxt = data[eg][0]["y"][idcs].unsqueeze(0)

In [ ]:
from ntbks_viz import plot_posterior_predefined_cntxt
plot_posterior_predefined_cntxt(data_trainers[3]['50%har/transformer_gnp_large'].module_, 
                                true_func=(data[eg][0]["X"].unsqueeze(0), data[eg][0]["y"].unsqueeze(0)),
                               X_cntxt=X_cntxt, 
                               Y_cntxt=Y_cntxt,
                               n_trgt=128,
                               n_samples=1,
                               is_plot_std=True,
                               train_min_max=(-1,1),
                               title="Posterior Samples Conditioned on {} Context Points ".format(10),
                                       y_idx=2)

In [ ]:
from skopt.space import Real, Categorical, Integer

params = {
    'module__x_dim': [X_DIM],
    'module__y_dim': [Y_DIM],
    'module__r_dim': [8,16,32,64],
    #'module__keys_to_tmp_attn__RadialBasisFunc': [MlpRBF, GaussianRBF],
    #'module__TmpSelfAttn__is_depth_separable': Categorical([True,False]),
}

def scorer(model, X, y=None):
    score = 0
    n_steps = 0
    
    dataset = model.get_dataset(X, y)

    # use th eloss as metric (i.e. return log likelihood)
    model.criterion_.is_use_as_metric = True
    
    valid_batch_count = 0
    for Xi, yi in model.get_iterator(dataset, training=False):
        n_steps += 1
        yi_res = yi 
        step = model.validation_step(Xi, yi)
        score += step["loss"].item()
        
    score /= n_steps
    
    return score

gs = BayesSearchCV(model, 
                   params, 
                   refit=False, 
                   cv=3, 
                   scoring=scorer)

gs.fit(skorch.helper.SliceDataset(data_train))
print(gs.best_score_, gs.best_params_)